# CS4001/4042 Assignment 1, Part B, Q2
In Question B1, we used the Category Embedding model. This creates a feedforward neural network in which the categorical features get learnable embeddings. In this question, we will make use of a library called Pytorch-WideDeep. This library makes it easy to work with multimodal deep-learning problems combining images, text, and tables. We will just be utilizing the deeptabular component of this library through the TabMlp network:

In [4]:
!pip install pytorch-widedeep


In [5]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from pytorch_widedeep.preprocessing import TabPreprocessor
from pytorch_widedeep.models import TabMlp, WideDeep
from pytorch_widedeep import Trainer
from pytorch_widedeep.metrics import R2Score


>Divide the dataset (‘hdb_price_prediction.csv’) into train and test sets by using entries from the year 2020 and before as training data, and entries from 2021 and after as the test data.

In [6]:
num_features = [
    "dist_to_nearest_stn",
    "dist_to_dhoby",
    "degree_centrality",
    "eigenvector_centrality",
    "remaining_lease_years",
    "floor_area_sqm",
]

cat_features = [
    "month",
    "town",
    "flat_model_type",
    "storey_range",
]

features = num_features + cat_features

targets = ["resale_price"]

df = pd.read_csv("hdb_price_prediction.csv")

df_train = df[df["year"] <= 2020]
df_test = df[df["year"] >= 2021]

train = df_train[features + targets]
test = df_test[features + targets]


>Refer to the documentation of Pytorch-WideDeep and perform the following tasks:
https://pytorch-widedeep.readthedocs.io/en/latest/index.html
* Use [**TabPreprocessor**](https://pytorch-widedeep.readthedocs.io/en/latest/examples/01_preprocessors_and_utils.html#2-tabpreprocessor) to create the deeptabular component using the continuous
features and the categorical features. Use this component to transform the training dataset.
* Create the [**TabMlp**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/model_components.html#pytorch_widedeep.models.tabular.mlp.tab_mlp.TabMlp) model with 2 linear layers in the MLP, with 200 and 100 neurons respectively.
* Create a [**Trainer**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/trainer.html#pytorch_widedeep.training.Trainer) for the training of the created TabMlp model with the root mean squared error (RMSE) cost function. Train the model for 100 epochs using this trainer, keeping a batch size of 64. (Note: set the *num_workers* parameter to 0.)

In [12]:
tab_preprocessor = TabPreprocessor(
    embed_cols=cat_features, continuous_cols=num_features
)
X_tab = tab_preprocessor.fit_transform(train)

tab_mlp = TabMlp(
    column_idx=tab_preprocessor.column_idx,
    cat_embed_input=tab_preprocessor.cat_embed_input,
    continuous_cols=num_features,
    mlp_hidden_dims=[200, 100],
)
model = WideDeep(deeptabular=tab_mlp)

trainer = Trainer(
    model,
    objective="root_mean_squared_error",
    num_workers=0,
    seed=SEED,
    device="cuda",
)
trainer.fit(
    X_tab=X_tab, target=train["resale_price"].values, n_epochs=100, batch_size=64
)


c:\Users\tonyh\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_widedeep\preprocessing\tab_preprocessor.py:334: UserWarning: Continuous columns will not be normalised
  warnings.warn("Continuous columns will not be normalised")
epoch 100: 100%|██████████| 1366/1366 [00:10<00:00, 133.58it/s, loss=4.99e+4]


>Report the test RMSE and the test R2 value that you obtained.

In [17]:
r2_score = R2Score()

X_tab_te = tab_preprocessor.transform(test)
y_pred = trainer.predict(X_tab=X_tab_te)

y_test = np.array(test["resale_price"])
y_pred = np.array(y_pred)

rmse = np.sqrt(np.sum((y_test - y_pred) ** 2) / len(y_test))
r2 = r2_score(y_pred, y_test)

print(f"Test RMSE: {rmse}")
print(f"Test R2: {r2}")


predict: 100%|██████████| 1128/1128 [00:03<00:00, 359.93it/s]

Test RMSE: 97072.19440824342
Test R2: 0.6707784730545674
